# 2.5 시퀀스에 덧셈과 곱셈 연산자 사용하기

`a`가 가변 항목을 담고 있을 때 `a * n`과 같은 표현식을 사용하면 원치 않은 결과가 나올 수 있음에 유의해야 한다.

In [8]:
my_list = [[]] * 3
my_list

[[], [], []]

In [10]:
id(my_list[0])

140672057933696

In [11]:
id(my_list[1])

140672057933696

In [12]:
id(my_list[2])

140672057933696

In [13]:
my_list_test = [[], [], []]

In [14]:
id(my_list_test[0])

140672125394624

In [15]:
id(my_list_test[1])

140672125624256

In [16]:
id(my_list_test[2])

140672125623232

## 2.5.1 리스트의 리스트 만들기

리스트를 담고 있는 리스트를 초기화할 때에는 list comprehension을 사용하는 것이 좋다.

In [17]:
board = [['_'] * 3 for i in range(3)]
board

[['_', '_', '_'], ['_', '_', '_'], ['_', '_', '_']]

In [19]:
board[1][2] = 'X'
board

[['_', '_', '_'], ['_', '_', 'X'], ['_', '_', '_']]

이는 내부적으로 다음과 같이 동작한다.

In [23]:
board = []
for i in range(3):
    row = ['_'] * 3
    board.append(row)
board

[['_', '_', '_'], ['_', '_', '_'], ['_', '_', '_']]

동일한 리스트에 대한 3개의 참조를 가진 리스트는 쓸모없다. 아래는 최상위 리스트가 동일한 내부 리스트에 대한 참조 3개를 가진다.

In [20]:
weird_board = [['_'] * 3] * 3
weird_board

[['_', '_', '_'], ['_', '_', '_'], ['_', '_', '_']]

In [21]:
weird_board[1][2] = 'X'
weird_board

[['_', '_', 'X'], ['_', '_', 'X'], ['_', '_', 'X']]

이는 내부적으로 다음과 같이 동작한다.

In [24]:
board = []
row = ['_'] * 3

for i in range(3):
    board.append(row)
board

[['_', '_', '_'], ['_', '_', '_'], ['_', '_', '_']]

# 2.6 시퀀스의 복합 할당

`+=` 연산자가 작동하도록 만드는 특수 메서드는 `__iadd__()` 인데, 이것이 구현되어 있지 않으면 파이썬은 대신 `__add__()` 메서드를 호출한다.     
- `__iadd__()` : 기존 객체의 내용 변경 (for 가변 객체)
- `__add__()` : `a = a + b`가 되어 1) `a + b` 계산해서 2) 새 객체 `a` 만들어 할당     

일반적으로 가변 시퀀스에 대해서는 `__iadd__()` 메서드를 구현할 것

In [25]:
l = [1, 2, 3]
id(l)

140671693788672

In [26]:
l *= 2
l

[1, 2, 3, 1, 2, 3]

In [27]:
id(l)

140671693788672

가변 시퀀스인 list에서는 `__iadd__()` 가 호출되어 기존 객체의 내용이 변경되었다. (메모리 주소 변하지 않음)

In [28]:
t = (1, 2, 3)
id(t)

140671694353152

In [29]:
t *= 2
t

(1, 2, 3, 1, 2, 3)

In [30]:
id(t)

140671690292960

불변 시퀀스인 tuple에서는 `__add__()` 가 호출되었다. (메모리 주소 변함)

## 2.6.1 `+=` 복합 할당 퀴즈

In [31]:
t = (1, 2, [30, 40])
t[2] += [50, 60]

TypeError: 'tuple' object does not support item assignment

In [32]:
t

(1, 2, [30, 40, 50, 60])

가변 항목을 튜플에 넣는 것은 좋은 생각이 아니다.

# 2.7 `list.sort()`와 `sorted()` 내장 함수

- `list.sort()` : 사본을 만들지 않고 리스트 내부를 변경해서 정렬
- `sorted()` : 새로운 리스트를 생성해서 반환

두 메서드가 받는 kwargs
- `reverse` : default = `False`이고 `True`이면 내림차순 정렬
- `key`: 정렬에 사용할 키를 생성하기 위해 각 항목에 적용할 함수. 예를 들어 문자열의 길이에 따라 정렬을 하고 싶다면 `key=len` 으로 지정할 것. default = 항목 자체에 대한 비교

In [33]:
fruits = ['grape', 'raspberry', 'apple', 'banana']
sorted(fruits)

['apple', 'banana', 'grape', 'raspberry']

In [34]:
sorted(fruits, reverse=True)

['raspberry', 'grape', 'banana', 'apple']

In [35]:
sorted(fruits, key=len)

['grape', 'apple', 'banana', 'raspberry']

In [36]:
sorted(fruits, key=len, reverse=True)

['raspberry', 'banana', 'grape', 'apple']

In [37]:
fruits

['grape', 'raspberry', 'apple', 'banana']

In [38]:
fruits.sort()

In [39]:
fruits

['apple', 'banana', 'grape', 'raspberry']

일단 시퀀스를 정렬한 후에는 효율적으로 검색할 수 있다.

# 2.8 정렬된 시퀀스를 bisect로 관리하기

- `bisect()` : 이진 검색 알고리즘을 이용해서 시퀀스 검색
- `insort()` : 정렬된 시퀀스 안에 항목 삽입

## 2.8.1 `bisect()`로 검색하기

In [50]:
import bisect
import sys

HAYSTACK = [1, 4, 5, 6, 8, 12, 15, 20, 21, 23, 23, 26, 29, 30]
NEEDLES = [0, 1, 2, 5, 8, 10, 22, 23, 29, 30, 31]

ROW_FMT = '{0:2d} @ {1:2d}  {2}{0:<2d}'

def demo(bisect_fn):
    for needle in reversed(NEEDLES):
        position = bisect_fn(HAYSTACK, needle)
        offset = position * '    |'
        print(ROW_FMT.format(needle, position, offset))
        
if __name__ == '__main__':
    if sys.argv[-1] == 'left':
        bisect_fn = bisect.bisect_left
    else:
        bisect_fn = bisect.bisect

print('DEMO:', bisect_fn.__name__)
print('haystack ->', '   '.join('%2d' % n for n in HAYSTACK))

demo(bisect_fn)

DEMO: bisect_right
haystack ->  1    4    5    6    8   12   15   20   21   23   23   26   29   30
31 @ 14      |    |    |    |    |    |    |    |    |    |    |    |    |    |31
30 @ 14      |    |    |    |    |    |    |    |    |    |    |    |    |    |30
29 @ 13      |    |    |    |    |    |    |    |    |    |    |    |    |29
23 @ 11      |    |    |    |    |    |    |    |    |    |    |23
22 @  9      |    |    |    |    |    |    |    |    |22
10 @  5      |    |    |    |    |10
 8 @  5      |    |    |    |    |8 
 5 @  3      |    |    |5 
 2 @  1      |2 
 1 @  1      |1 
 0 @  0  0 


In [56]:
def grade(score, breakpoints=[60, 70, 80, 90], grades='FDCBA'):
    i = bisect.bisect(breakpoints, score)
    return grades[i]

[grade(score) for score in [33, 99, 77, 70, 89, 90, 100]]

['F', 'A', 'C', 'C', 'B', 'A', 'A']

정렬된 긴 숫자 시퀀스를 검색할 때 `index()`보다 `bisect()` 함수를 사용

## 2.8.2 `bisect.insort()`로 삽입하기

정렬에는 연산이 많이 필요하므로, 시퀀스를 일단 정렬한 후에는 정렬 상태를 유지하는 것이 좋다. 이를 위해 `bisect.insort()` 함수가 만들어졌다.    
`insort(seq, item)`은 `seq`을 오름차순으로 유지한 채로 `item`을 `seq`에 삽입한다.

In [57]:
import bisect
import random

SIZE = 7

random.seed(1729)
my_list = []
for i in range(SIZE):
    new_item = random.randrange(SIZE*2)
    bisect.insort(my_list, new_item)
    print('%2d ->' % new_item, my_list)

10 -> [10]
 0 -> [0, 10]
 6 -> [0, 6, 10]
 8 -> [0, 6, 8, 10]
 7 -> [0, 6, 7, 8, 10]
 2 -> [0, 2, 6, 7, 8, 10]
10 -> [0, 2, 6, 7, 8, 10, 10]


# 2.9 리스트가 답이 아닐 때

리스트가 사용하기 편하지만, 세부 요구사항에 따라 더 나은 자료형도 있다.
- 많은 실수를 저장해야 하는 경우 : 배열
- 리스트 양쪽 끝에 항목을 계속 추가하거나 삭제해야 한다면 : 덱

## 2.9.1 배열

리스트 안에 숫자만 들어 있다면 배열이 리스트보다 더 효율적이다. 배열은 `pop()`, `insert()`, `extend()` 등을 포함해서 가변 시퀀스가 제공하는 모든 연산을 지원하면서 C 배열만큼 가볍다.
- 숫자가 많이 들어있는 시퀀스의 경우, 배열에 저장하면 메모리가 많이 절역된다.
- 배열형에 맞지 않는 숫자는 저장할 수 없다.

In [58]:
from array import array
from random import random

floats = array('d', (random() for i in range(10**7)))
floats[-1]

0.5963321947530882

In [60]:
fp = open('floats.bin', 'wb')
floats.tofile(fp)
fp.close()

In [62]:
floats2 = array('d')
fp = open('floats.bin', 'rb')
floats2.fromfile(fp, 10**7)
fp.close()
floats2[-1]

0.5963321947530882

In [63]:
floats2 == floats

True

## 2.9.2 메모리 뷰

byte를 복사하지 않고 배열의 슬라이스를 다룰 수 있는 방법

In [65]:
import array
numbers = array.array('h', [-2, -1, 0, 1, 2])
memv = memoryview(numbers)
len(memv)

5

In [66]:
memv[0]

-2

In [67]:
memv_oct = memv.cast('B')
memv_oct.tolist()

[254, 255, 255, 255, 0, 0, 1, 0, 2, 0]

In [68]:
memv_oct[5] = 4

In [70]:
numbers

array('h', [-2, -1, 1024, 1, 2])

## 2.9.3 NumPy와 SciPy

In [71]:
import numpy as np
a = np.arange(12)
a

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11])

In [72]:
type(a)

numpy.ndarray

In [73]:
a.shape

(12,)

In [74]:
a.shape = 3, 4
a

array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11]])

In [75]:
a[2]

array([ 8,  9, 10, 11])

In [76]:
a[2,1]

9

In [77]:
a[:, 1]

array([1, 5, 9])

In [78]:
a.transpose()

array([[ 0,  4,  8],
       [ 1,  5,  9],
       [ 2,  6, 10],
       [ 3,  7, 11]])

## 2.9.4 덱 및 기타 큐

- `append()`와 `pop()` 메서드를 사용해서 리스트를 스택이나 큐로 사용할 수 있다.
- `collections.deque` 은 큐의 양쪽 어디에서든 빠르게 삽입 및 삭제가 가능하다.

In [83]:
from collections import deque
dq = deque(range(10), maxlen=10)
dq

deque([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [84]:
dq.rotate(3)
dq

deque([7, 8, 9, 0, 1, 2, 3, 4, 5, 6])

In [85]:
dq.rotate(-4)
dq

deque([1, 2, 3, 4, 5, 6, 7, 8, 9, 0])

In [86]:
dq.appendleft(-1)
dq

deque([-1, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [87]:
dq.extend([11, 22, 33])
dq

deque([3, 4, 5, 6, 7, 8, 9, 11, 22, 33])

In [88]:
dq.extendleft([10, 20, 30, 40])
dq

deque([40, 30, 20, 10, 3, 4, 5, 6, 7, 8])

# 3.1 일반적인 매핑형

함수 인수가 `dict`인지 검사하는 것보다 `isinstance()` 함수를 사용하는 것이 더 좋다. (값은 해시 가능할 필요 없고, 키만 해시 가능하면 된다)

'해시 가능하다' 의 의미는?
- 수명 주기 동안 결코 변하지 않는 해시값을 가지고 있고
- 다른 객체와 비교할 수 있는      

객체

In [2]:
tt = (1, 2, (30, 40))
hash(tt)

-3907003130834322577

In [3]:
tl = (1, 2, [30, 40])
hash(tl)

TypeError: unhashable type: 'list'

In [4]:
tf = (1, 2, frozenset([30, 40]))
hash(tf)

5149391500123939311

해시와 id는 같은 것을 의미하는 것인가?

In [10]:
a = dict(one=1, two=2, three=3)
b = {'one':1, 'two':2, 'three':3}
c = dict(zip(['one', 'two', 'three'], [1, 2, 3]))
d = dict([('two', 2), ('one', 1), ('three', 3)])
e = dict({'three':3, 'one':1, 'two':2})
a == b == c == d == e

True

# 3.2 지능형 딕셔너리

list comprehension과 제너레이터 표현식 구문이 dictionary comprehension에도 적용된다.    
dictionary comprehension은 모든 반복형 객체에서 키-값 쌍을 생성함으로써 딕셔너리 객체를 만들 수 있다.

In [12]:
DIAL_CODES = [
    (86, 'China'),
    (91, 'India'),
    (1, 'United States'),
    (62, 'Indonesia'),
    (55, 'Brazil'),
    (92, 'Pakistan'),
    (880, 'Bangladesh'),
    (234, 'Nigeria'),
    (7, 'Russia'),
    (81, 'Japan')
]
country_code = {country:code for code, country in DIAL_CODES}
country_code

{'China': 86,
 'India': 91,
 'United States': 1,
 'Indonesia': 62,
 'Brazil': 55,
 'Pakistan': 92,
 'Bangladesh': 880,
 'Nigeria': 234,
 'Russia': 7,
 'Japan': 81}

In [15]:
{code:country.upper() for country, code in country_code.items() if code < 66}

{1: 'UNITED STATES', 62: 'INDONESIA', 55: 'BRAZIL', 7: 'RUSSIA'}

# 3.3 공통적인 매핑 메서드

## 3.3.1 존재하지 않는 키를 `setdefault()`로 처리하기

존재하지 않는 키 `k`로 `d[k]`를 접근하면 dictionary는 오류를 발생시킨다. `KeyError`를 처리하는 것보다 기본값을 사용하는 것이 더 편리한 경우에는 `d[k]` 대신 `d.get(k, default)`을 사용한다.

In [17]:
country_code['Korea']

KeyError: 'Korea'

In [20]:
country_code.get('Korea')

In [21]:
country_code.get('China')

86

하지만 가변 객체에서 `__getitem__()`이나 `get()` 메서드는 사용하기 어색하다.

# 3.4 융통성 있게 키를 조회하는 매핑

## 3.4.1 `defaultdict` : 존재하지 않는 키에 대한 또 다른 처리

`defaultdict`는 존재하지 않는 키로 검색할 때 요청에 따라 `__getitem__()` 메서드를 호출할 때마다 기본값 항목을 생성한다.    
`default_factory`가 설정되어 있지 않으면 `KeyError`가 발생한다.

## 3.4.2 `__missing__()` 메서드

존재하지 않는 키를 처리하는 방법으로, 1) dict 클래스를 상속하고 2) `__missing__()` 메서드를 정의하면, `dict.__getitem__()`가 키를 발견할 수 없을 때 `KeyError`를 발생시키지 않고 `__missing__()` 메서드를 호출한다.

In [22]:
class StrKeyDict0(dict):
    
    def __missing__(self, key):
        if isinstance(key, str):
            raise KeyError(key)
        return self[str(key)]
    
    def get(self, key, default=None):
        try:
            return self[key]
        except KeyError:
            return default
    
    def __contains__(self, key):
        return key in self.keys() or str(key) in self.keys()

In [23]:
d = StrKeyDict0([('2', 'two'), ('4', 'four')])
d['2']

'two'

In [24]:
d[4]

'four'

In [25]:
d[1]

KeyError: '1'

In [26]:
d.get('2')

'two'

In [27]:
d.get(4)

'four'

In [28]:
d.get(1, 'N/A')

'N/A'

In [29]:
2 in d

True

In [30]:
1 in d

False

왜 `__missing__()` 메서드 안에 `isinstance(key, str)`가 필요한가? 이 부분이 없으면 `str(k)` 키가 존재하지 않는 경우 무한히 재귀적으로 호출된다.     

`__contains()__` 메서드는 왜 필요한가?

# 3.5 그 외 매핑형